# Notebook for analysing data to find potential signals within
# 

### How does this script work?
First saved data in .txt form is loaded in and the dataset is extracted. The dataset is then ran through a pipeline looking for potential signals within the data and finally Dataframes are created with the characteristics of the full dataset and any signals within
# 
### How does the pipeline work?
The data is first analysed my a machine learning model. This model is un-supervised and is looking for regions in the data where there is high volatility, indicating a potential region of interest where a signal may be.

The predictions made by the model are then looked at and any regions of interest are saved and looked into with more detail. This invloves using a cross correlation function to determine if the signal has the characteristic curve, if the extracted signal is extremely long then the cross correlation function is used to see if there is a signal within the region of interest. If the cross correlation functions show a signal is present then they are analysed looking at the length, number of peaks and height of the peaks as well as the depth of the dip. If the cross correlation function shows no signal is present then the region of interest is discarded.

In [ ]:
import MLDetectionFunctionsPythonV9_5_NoRestricts_MLcreate as EEGanalysis
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, Output, fixed, interact_manual
from IPython.display import display, clear_output
import numpy as np
import math
import mat73
import pickle

In [ ]:
#AMAZING IDEA

#What if I saved all individual areas of interest extracted by the model
#this includes seizure events and non-seizure events
#use this to train a model that can classify each extracted area of interest

#save both raw and smoothed AoI and see which has better results



#Try first - just key infor in dataframe to train with

#using the extracted dataframe at the end of the code use this with either 'Signal' or 'Noise'
#to train a model which can then be used to classify extracted AoI instead of full signals

#trim down dataframe so doesnt include confusing info eg start and end times
#intead just have signal length

#include cross-corr score as well in dataframe
#think of other things to input into the dataframe

#maybe use rounded signal length (in seconds) as well as rounded dip depth, medium peak height

#look at peak density throughout signal
#rolling average over 10secs and pick median, mean or peak value?





#this looks to be a very good idea

# Importing data to analyse

### Each dataset should be saved as a .txt file

In [ ]:
realdatafile1 = 'Long_Data_1.txt'
realdatafile2 = 'Long_Data_2.txt'
realdatafile3 = 'Long_Data_3.txt'
realdatafile4 = 'Long_Data_4.txt'

realdatafile5 = 'Short_Data_1.txt'

realdatafile6 = '22.07.22_Example_Data_Long.txt'
realdatafile7 = '22.07.22_Example_Data_Long2.txt'

realdatafile8 = '041022_Example_Data_Long_NoisyMW117CH1.txt'
realdatafile9 = '041022_Example_Data_Long_SpikesMW117CH3.txt'

realdatafile10 = '12.10.22_Example_Data_Long_Spikes-Venus16.3CH1.txt' #this is same as a previous file

realdatafile11 = '041022ExampleDataLongVenus13p1CH1.txt' #could be all noise as 'signals' all very short
realdatafile12 = '041022ExampleDataLongVenus13p1CH2.txt' #also very difficult to classify as nearly all could be signal or noise
realdatafile13 = '041122ExampleDataLongVenus7p3CH3.txt'
realdatafile14 = '041122ExampleDataLongVenus7p3CH4.txt'
realdatafile15 = '041122ExampleDataLongVenus10p1CH1.txt'
realdatafile16 = '041122ExampleDataLongVenus10p1CH2.txt'
realdatafile17 = '041122ExampleDataLongVenus12p3CH1.txt'
realdatafile18 = '041122ExampleDataLongVenus12p3CH2.txt'
realdatafile19 = '041122ExampleDataLongVenus13p3CH1.txt' #no extractions
realdatafile20 = '041122ExampleDataLongVenus13p3CH2.txt' #no extractions

#start of novel analysis
realdatafile21 = '21.11./mnt/datafast/Douglas/MaxTransfer/Data040523/01.03.22 - MW109 - CH1 - 0 Mg.txt23 - MW3 - CH2.txt'
realdatafile22 = '21.12.02 - MW30 - CH3.txt'
realdatafile23 = '22.01.24 - MW63 - CH2.txt' #unique error
realdatafile24 = '22.01.24 - MW65 - CH1.txt'
realdatafile25 = '22.01.21 - MW61 - CH2.txt'
realdatafile26 = '21.12.14 - MW51 - CH2.txt'
realdatafile27 = '21.12.15 - MW58 - CH3.txt' #lots of very smooth signals, well detected and classified


noveldata1 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.23 - MW3 - CH2.txt' #some signals, extractions possibly too long
noveldata2 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.23 - MW5 - CH2.txt' #unique error FIXED. 2 extractions, both seem to be calibration noise classified as signal
noveldata3 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.25 - MW7 - CH1.txt' #single extraction, noise
noveldata4 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.25 - MW9 - CH2.txt' #same unique error FIXED. about 10 extractions, all noise except a few calibrations classified as signal
noveldata5 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.29 - MW13 - CH3.txt' #one extraction, noise
noveldata6 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.30 - MW19 - CH2.txt' #3 extracts, all noise
noveldata7 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.11.30 - MW19 - CH4.txt' #no extracts
noveldata8 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW21 - CH4.txt' #3 extracts, all noise
noveldata9 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW25 - CH1.txt' #4 signals extracted, last one possible starting too early
noveldata10 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW29 - CH1.txt' #4 extracts, 1/2 disputable signals
noveldata11 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW29 - CH2.txt' #4 extracts, all noise
noveldata12 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW30 - CH3.txt' #1 extract, noise
noveldata13 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.02 - MW32 - CH4.txt' #no extractions
noveldata14 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.06 - MW33 - CH2.txt' #4 extracts. all noise but 1 wrongly classified as signal
noveldata15 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.06 - MW35 - CH1.txt' #4 extracts, all noise
noveldata16 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.06 - MW36 - CH4.txt' #1 extract, noise
noveldata17 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.08 - MW41 - CH1.txt' #v strong signals. 4 extracts, 3 clear signals, 1 possible noise/signal
noveldata18 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.08 - MW42 - CH3.txt' #again, some good signals. 7 extractions, some signal and noise. all well classified
noveldata19 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.09 - MW46 - CH4.txt' #dubios signal extractions. look to be a rig calibration
noveldata20 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.14 - MW51 - CH2.txt' #8 extracts, mainly noise with 1 possible signal
noveldata21 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.14 - MW53 - CH1.txt'#mega dataset size. lots of extractions 30+, many signals and noise
noveldata22 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.14 - MW54 - CH3.txt'#mega dataset size, 40+ extractions. lots of signals and noise
noveldata23 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.15 - MW58 - CH3.txt'#nice example dataset. lots of very well extracted signals
noveldata24 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/21.12.15 - MW58 - CH4.txt'#struggles with correct detection (could make for good training data). lots of single dips classified as signal
noveldata25 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.21 - MW61 - CH2.txt'#lots of extractions, many noise and some signals. signals are of interesting shape
noveldata26 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.24 - MW63 - CH1.txt'#generally good extraction, maybe cuts some signals short
noveldata27 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.24 - MW63 - CH2.txt'#a few nice signals but also some noise (possibly some misclassified noise)
noveldata28 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.24 - MW64 - CH3.txt' # lots of signals and noise, some possibly misclassified
noveldata29 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.24 - MW65 - CH1.txt'#only 2 extractions, both nosie
noveldata30 = '/mnt/datafast/Douglas/MaxTransfer/NewDataToAnalyse/22.01.24 - MW65 - CH2.txt' #7 extractions, all noise but 2 possible signals
#re-run all novel data as MainDataframe wasnt sorted correctly and could have incorrect predictions for the signal number
#signals that were X.1 X.2 might not be sorted correctly


realdatafile21van = '2022_09_30_socialtaskephys_exp_SGHRB_subject2777_Int16ch5K0_I2C625Hz_1ADC200K_IR320K.edf'
realdatafile22van = '2022_09_30_socialtaskephys_exp_SGHRB_subject2777_Int16ch5K0_I2C625Hz_1ADC200K_IR320K.mat'




curvedata = 'SpikeWaveDischarge.txt'

#add start and end vals to compare with Max spreadsheet into dataframe

In [ ]:
datamay1 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/01.03.22 - MW109 - CH1 - 0 Mg.txt'
datamay2 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/01.03.22 - MW109 - CH2 - 0 Mg.txt'
datamay3 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/01.03.22 - MW110 - CH3 - 0 Mg.txt'
datamay4 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/01.03.22 - MW110 - CH4 - 0 Mg.txt'
datamay5 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/02.02.22 - MW71 - CH2 - 4-AP.txt'
datamay6 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW73 - CH1 - 4-AP.txt'
datamay7 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW73 - CH2 - 4-AP.txt' #a few too long extractions
datamay8 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW74 - CH3 - 4-AP.txt' #a few too long extractions
datamay9 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW74 - CH4 - 4-AP.txt' #fairly good
datamay10 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW75 - CH1 - 4-AP.txt' #Very good example
datamay11 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW75 - CH2 - 4-AP.txt' #good extractions, nice example
datamay12 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW76 - CH3 - 4-AP.txt' #lots of signals, unique look to them
datamay13 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW76 - CH4 - 4-AP.txt' #lots od extracts, some boarderline singla/noise and plenty singal
datamay14 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/03.02.22 - MW77 - CH2 - 0 Mg.txt' #only 3 signals but good extracts
datamay15 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/07.02.22 - MW79 - CH1 - 4-AP.txt' #nice clean extractions, 1 maybe too early
datamay16 = '/mnt/datafast/Douglas/MaxTransfer/Data040523/07.02.22 - MW79 - CH2 - 4-AP.txt' #lots of clean extractions, maybe 2 too early

# Choosing which dataset to analyse

In [ ]:
datasetusing = datamay7

#check 5, boarderline on signal/noise (especially 3rd signal)

#8 is good for testing with 'clean' dataset that works v well with original 9_5 version

#11 has detection issues
#14 is cool as works well
#15 cool too
#16 is v good
#17 detection issues due to dip depth too small and thus removing signal


In [ ]:
realdatausing = EEGanalysis.getdata(datasetusing)
realdatausing

In [ ]:
DFnorm, signal = EEGanalysis.CreateDataframe(realdatausing)
DFnormres, signalres2 = EEGanalysis.CreateDataframe(realdatausing)

# Formatting the curve data to use in cross correlation to determine validity of potential signals

In [ ]:
curvedata = EEGanalysis.getdata(curvedata)

In [ ]:
timecurve, signalcurve = EEGanalysis.timesignal(curvedata)

In [ ]:
signalcurveave = EEGanalysis.movingaverage(signalcurve[:,1],10000)

# Predicting the locations of potential signals within the dataset

In [ ]:
StartVals, EndVals, signalCURVES = EEGanalysis.PredictSignal(DFnorm, signal,350000)

# Analysing the whole dataset before and potential signals have been removed

In [ ]:
MainSignalDFtestfull = EEGanalysis.AnalyseSignalFullData(signal)

# Extracting potential individual signals, determining their validity and analysing them where necessary

In [ ]:
MainSignalDF, MainSignalDFraw, MainSignalDFccextracts,redsig = EEGanalysis.AnalyseSignalsFailsafeVer(StartVals,EndVals,signal,signalres2,signalCURVES,DFnorm,signalcurveave,250000)


# Analysing the remaining data after valid signals have been removed

In [ ]:
MainSignalDFtestfullthird = EEGanalysis.AnalyseSignalFullData(redsig)

# Dataframe containing characteristics of full original data

In [ ]:
MainSignalDFtestfull

# Dataframe containing characteristics of initially extracted signals, not including any second tier extractions

In [ ]:
MainSignalDFraw

# Dataframe containing characteristics of only second tier extractions

### This is where a signal was within an initial extraction but the original signal contained more data then just the signal

### The signal is extracted using cross correlation to trim out the 'excess' data from the original extraction

In [ ]:
MainSignalDFccextracts

# Dataframe containing the characteristics of the full data with signals removed

In [ ]:
MainSignalDFtestfullthird

# Combining the dataframes containing original extracted signals and second tier extractions

### Original extractions are labelled with .0 i.e. 5.0 or 8.0

### Second tier extractions are labelled with .1 .2 .3 depending on how many second tier extractions there are within one original signal  i.e original signal 6 may have two signals within it and they would be labelled 6.1 and 6.2   original signal 9 may have one signal within it and would be labelled 9.1

In [ ]:
MainSignalDFtot = MainSignalDFraw.append(MainSignalDFccextracts, ignore_index=True)


In [ ]:
MainSignalDFtot['SignalNumber'] = MainSignalDFtot['SignalNumber'].str.split('Signal').str.join('')

MainSignalDFtot['SignalNumber'] = pd.to_numeric(MainSignalDFtot['SignalNumber'])

MainSignalDFtot=MainSignalDFtot.sort_values('SignalNumber')


In [ ]:
MainSignalDF = MainSignalDF.rename(columns={"Startsig": "Start Time (s)", "Endsig": "End Time (s)"
                                           , "SignalLength": "Signal Length (s)"
                                           , "NoPeaksBeforeGlobal": "Peaks Before Global Min"
                                           , "NoPeaksAfterGlobal": "Peaks After Global Min"
                                           , "NoDipsBeforeGlobal": "Dips Before Global Min"
                                           , "NoDipsAfterGlobal": "Dips After Global Min"
                                           , "DipDepth": "Global Min Depth"
                                           , "Start": "Start Index"
                                           , "End": "End Index"})

In [ ]:
MainSignalDF

In [ ]:
#MainSignalDF['SignalNumber'] = [1,2,3,4,5]
#MainSignalDF

In [ ]:
#MainSignalDF.to_csv('Dataset7Output.csv', index = False)

In [ ]:
realdataV2 = EEGanalysis.getdata(datasetusing)
DFnorm3, signalres3 = EEGanalysis.CreateDataframe(realdataV2)

In [ ]:
MainSignalDF = MainSignalDF.fillna(0)
MainSignalDF

In [ ]:
MainSignalDF = MainSignalDF.sort_index()
MainSignalDF

In [ ]:
#MainSignalDF = MainSignalDF.drop([1])
#MainSignalDF

In [ ]:
MainSignalDFpred = MainSignalDF.sort_index()

In [ ]:
MainSignalDFpred = MainSignalDFpred.astype(float).round(0).astype(int)


In [ ]:
MainSignalDFpred = MainSignalDFpred.drop('Start Time (s)', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('End Time (s)', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('Start Index', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('End Index', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('SignalNumber', axis=1)
MainSignalDFpred

In [ ]:
MLclassifier = pickle.load(open('PotentialSignalClassifier.sav', 'rb'))

In [ ]:
MainSignalDFpredar = MainSignalDFpred.to_numpy()

In [ ]:
prediction = MLclassifier.predict(MainSignalDFpredar)

In [ ]:
prediction

In [ ]:
print('All Curves Overlaid:')
for a in range(len(MainSignalDF)):
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])

In [ ]:
print('All Curves Staggared:')
for a in range(len(MainSignalDF)):
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1]-2000*(a+1))

In [ ]:
for a in range(len(MainSignalDF)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%MainSignalDFpred['Signal Length (s)'][a])
    print(prediction[a])
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    plt.show()
    print()
    print()

In [ ]:
for a in range(len(MainSignalDF)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%MainSignalDFpred['Signal Length (s)'][a])
    print('Prediction:%s' %prediction[a])
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    plt.show()
    
    def f(Signal):
            1+1

    #locals()["my_result"+str(a)] = interactive(f, Signal=True)
    locals()["my_result"+str(a)] = widgets.RadioButtons(options=['Signal', 'Noise'],description='Signal or Noise:',disabled=False)

    display(locals()["my_result"+str(a)])
    
    locals()["UnuseableWhy"+str(a)] = input("If Signal Unuseable, Why? (Leave Blank and Press Enter if Useable)")
    
    
    
    input("Press Enter to Confirm")
    

    print()
    print()
    clear_output()

In [ ]:
for i in range(len(MainSignalDF)):
    print(locals()["my_result"+str(i)].value)

In [ ]:
MainSignalDF['User Entry'] = np.nan
MainSignalDF['Unuseable Reason'] = np.nan

for i in range(len(MainSignalDF)):
    MainSignalDF['User Entry'][i] = locals()["my_result"+str(i)].value
        
for i in range(len(MainSignalDF)):
        MainSignalDF['Unuseable Reason'][i] = locals()["UnuseableWhy"+str(i)]

In [ ]:
MainSignalDF

In [ ]:
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        MainSignalDF = MainSignalDF.drop([a])
        
MainSignalDF = MainSignalDF.reset_index()

In [ ]:
MainSignalDF

In [ ]:
globals()[iter] = 0

In [ ]:
DropdownOptions = []
for i in range(len(MainSignalDF)):
    DropdownOptions.append(('Signal %s'%str(i+1), i))

# Run the following cell two times, and interact with it the on the second run

In [ ]:
%matplotlib notebook
from numpy import pi, sin
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons


SignalSelector = widgets.Dropdown(
    options=DropdownOptions,
    value=globals()[iter],
    description='Signal:',
)

def reset_button_on_clicked(mouse_event):
    start_slider.reset()
    end_slider.reset()
    
def confirm_button_on_clicked(mouse_event):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    highlightzone = ax.axvspan(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='green', alpha=0.25)
    fig.canvas.draw_idle()
    globals()["StartCrop"+str(globals()[iter])] = start_slider.val
    globals()["EndCrop"+str(globals()[iter])] = end_slider.val
    
    
def sliders_on_changedstart(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
def sliders_on_changedend(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
    
x = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,0]

y = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,1]
    


axis_color = 'lightgoldenrodyellow'

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)

fig.subplots_adjust(left=0.25, bottom=0.25)

[line] = ax.plot(x, y, linewidth=0.5, color='black')



start_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03], facecolor=axis_color)
start_slider = Slider(start_slider_ax, 'Start', 0, 100, valinit=0)

end_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03], facecolor=axis_color)
end_slider = Slider(end_slider_ax, 'End', 0, 100, valinit=0)

#highlightplot = ax.axvspan(start_slider.val/2, 100 - end_slider.val/2, color='green', alpha=0.5)
highlightplotstart = ax.axvline(np.min(x), color='green', zorder=-1)
highlightplotend = ax.axvline(np.max(x), color='red', zorder=-1)
    
start_slider.on_changed(sliders_on_changedstart)
end_slider.on_changed(sliders_on_changedend)

reset_button_ax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
reset_button = Button(reset_button_ax, 'Reset', color='red', hovercolor='0.975')

confirm_button_ax = fig.add_axes([0.25, 0.025, 0.1, 0.04])
confirm_button = Button(confirm_button_ax, 'Confirm', color='green', hovercolor='0.975')
    
reset_button.on_clicked(reset_button_on_clicked)
confirm_button.on_clicked(confirm_button_on_clicked)

    
plt.show()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])
        globals()[iter] = change['new']
        
        %rerun
       

SignalSelector.observe(on_change)

display(SignalSelector)

In [ ]:
SignalCropPercs = pd.DataFrame(columns = ['Signal', 'Start Perc', 'End Perc'])
Signallist = []
Startlist = []
Endlist = []
for i in range(len(MainSignalDF)):
    Signallist.append('Signal %s'%str(i+1))
    Startlist.append(globals()["StartCrop"+str(i)])#SignalCropPercs['Start Perc'][i] = ['StartCrop%s'%i]
    Endlist.append(globals()["EndCrop"+str(i)])#SignalCropPercs['End Perc'][i] = ['EndCrop%s'%i]
    
SignalCropPercs['Signal'] = Signallist
SignalCropPercs['Start Perc'] = Startlist
SignalCropPercs['End Perc'] = Endlist

SignalCropPercs

In [ ]:
MainSignalDF

In [ ]:
StartIndice = MainSignalDF['Start Time (s)']*5000
EndIndice = MainSignalDF['End Time (s)']*5000
SignalLengthIndice = EndIndice - StartIndice
SignalMidpointIndice = SignalLengthIndice/2

In [ ]:
StartIndice

In [ ]:
EndIndice

In [ ]:
SignalLengthIndice/5000

In [ ]:
SignalMidpointIndice/5000

In [ ]:
CropStartIndice = StartIndice + (SignalCropPercs['Start Perc']/100)*SignalMidpointIndice
#100% = midpoint
#0% = startpoint

In [ ]:
CropEndIndice = EndIndice - (SignalCropPercs['End Perc']/100)*SignalMidpointIndice

In [ ]:
CropStartIndice

In [ ]:
CropEndIndice

In [ ]:
CropSignalTEST = pd.DataFrame(columns = ['Start', 'End'])
CropSignalTEST['Start'] = CropStartIndice/5000
CropSignalTEST['End'] = CropEndIndice/5000
CropSignalTEST

In [ ]:
%matplotlib inline
for a in range(len(CropSignalTEST)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%(int(CropSignalTEST['End'][a])-int(CropSignalTEST['Start'][a])))
    plt.plot(signalres3[int(CropSignalTEST['Start'][a]*5000):int(CropSignalTEST['End'][a]*5000),1])
    plt.show()
    print()
    print()

In [ ]:

MainSignalDFTEST = EEGanalysis.AnalyseSignalsCROPPED(CropSignalTEST['Start']*5000,CropSignalTEST['End']*5000,signalres3)

In [ ]:
MainSignalDFTEST

In [ ]:
len(signalres3[int(CropSignalTEST['Start'][0]*5000):int(CropSignalTEST['End'][0]*5000),1])

# GOT UP TO HERE

In [ ]:
# Know raw extraction start and end times (so also know length)

#know percentage to halfway from start/end for human start and end

#work out the true start/end from raw extraction and human percentages




#write function that takes these start and ends, along with the whole signal to analyse these start and end points as signals

# Quality Control

In [ ]:
#make whole notebook/script runnable on eddie 

#will need to remove displaying graphs and increactive QC
#can keep loading bars though

#can try as both individual functions and a full pipeline

#need to work out how much memory is required (cpu corse does not need to be massive, just need suitable RAM)

In [ ]:
signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000]

In [ ]:
globals()[iter] = 0

In [ ]:
DropdownOptions = []
for i in range(len(MainSignalDF)):
    DropdownOptions.append(('Signal %s'%str(i+1), i))

# Run the following cell two times, and interact with it the on the second run

In [ ]:
%matplotlib notebook
from numpy import pi, sin
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons


SignalSelector = widgets.Dropdown(
    options=DropdownOptions,
    value=globals()[iter],
    description='Signal:',
)

def reset_button_on_clicked(mouse_event):
    start_slider.reset()
    end_slider.reset()
    
def confirm_button_on_clicked(mouse_event):
    globals()["StartCrop"+str(globals()[iter])] = start_slider.val
    globals()["EndCrop"+str(globals()[iter])] = end_slider.val
    
    
def sliders_on_changedstart(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
def sliders_on_changedend(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
    
x = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,0]

y = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,1]
    


axis_color = 'lightgoldenrodyellow'

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)

fig.subplots_adjust(left=0.25, bottom=0.25)

[line] = ax.plot(x, y, linewidth=0.5, color='black')



start_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03], facecolor=axis_color)
start_slider = Slider(start_slider_ax, 'Start', 0, 100, valinit=0)

end_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03], facecolor=axis_color)
end_slider = Slider(end_slider_ax, 'End', 0, 100, valinit=0)

#highlightplot = ax.axvspan(start_slider.val/2, 100 - end_slider.val/2, color='green', alpha=0.5)
highlightplotstart = ax.axvline(np.min(x), color='green', zorder=-1)
highlightplotend = ax.axvline(np.max(x), color='red', zorder=-1)
    
start_slider.on_changed(sliders_on_changedstart)
end_slider.on_changed(sliders_on_changedend)

reset_button_ax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
reset_button = Button(reset_button_ax, 'Reset', color='red', hovercolor='0.975')

confirm_button_ax = fig.add_axes([0.25, 0.025, 0.1, 0.04])
confirm_button = Button(confirm_button_ax, 'Confirm', color='green', hovercolor='0.975')
    
reset_button.on_clicked(reset_button_on_clicked)
confirm_button.on_clicked(confirm_button_on_clicked)

    
plt.show()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])
        globals()[iter] = change['new']
        
        %rerun
       

SignalSelector.observe(on_change)

display(SignalSelector)

In [ ]:
SignalCropPercs = pd.DataFrame(columns = ['Signal', 'Start Perc', 'End Perc'])
Signallist = []
Startlist = []
Endlist = []
for i in range(len(MainSignalDF)):
    Signallist.append('Signal %s'%str(i+1))
    Startlist.append(globals()["StartCrop"+str(i)])#SignalCropPercs['Start Perc'][i] = ['StartCrop%s'%i]
    Endlist.append(globals()["EndCrop"+str(i)])#SignalCropPercs['End Perc'][i] = ['EndCrop%s'%i]
    
SignalCropPercs['Signal'] = Signallist
SignalCropPercs['Start Perc'] = Startlist
SignalCropPercs['End Perc'] = Endlist

SignalCropPercs

In [ ]:

for a in range(len(MainSignalDF)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%MainSignalDFpred['Signal Length (s)'][a])
    print('Prediction:%s' %prediction[a])
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    plt.show()
    
    def f(Signal):
            1+1

    #locals()["my_result"+str(a)] = interactive(f, Signal=True)
    locals()["my_result"+str(a)] = widgets.RadioButtons(options=['Signal', 'Noise'],description='Signal or Noise:',disabled=False)

    display(locals()["my_result"+str(a)])
    
    locals()["UnuseableWhy"+str(a)] = input("If Signal Unuseable, Why? (Leave Blank and Press Enter if Useable)")
    
    
    
    input("Press Enter to Confirm")
    

    print()
    print()
    clear_output()
    
    #change Signal Button to  radio Signal or Noise Buttons

In [ ]:
for i in range(len(MainSignalDF)):
    print(locals()["my_result"+str(i)].value)

In [ ]:
MainSignalDF['User Entry'] = np.nan
MainSignalDF['Unuseable Reason'] = np.nan

for i in range(len(MainSignalDF)):
    MainSignalDF['User Entry'][i] = locals()["my_result"+str(i)].value
        
for i in range(len(MainSignalDF)):
        MainSignalDF['Unuseable Reason'][i] = locals()["UnuseableWhy"+str(i)]

In [ ]:
MainSignalDF

In [ ]:
print('All Curves Overlaid:')
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        continue
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    
#do another plot with overlaid and staggered all centered on global minimum of each curve

In [ ]:
print('All Curves Overlaid:')
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        continue
        
    argmin = np.argmin(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] = range(0,len(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0]))
    signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] = signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] - argmin
    
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0],signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])

In [ ]:
print('All Curves Staggared:')
count = 0
offset = 4000
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        continue
    
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1]-offset*(count+1))
    count = count+1

In [ ]:
print('All Curves Staggared:')
count = 0
offset = 4000
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        continue
    
    argmin = np.argmin(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] = range(0,len(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0]))
    signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] = signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0] - argmin
    
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,0],signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1]-offset*(count+1))
    count = count+1

In [ ]:
MainSignalDF['Predicted Type'] = prediction
MainSignalDF

In [ ]:
savepath = datasetusing.replace('.txt','')
savepath

In [ ]:
MainSignalDF.to_excel(savepath + ' - SPREADSHEET.xlsx', index = False)
